In [ ]:
#Desafio BB
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import zipfile

if(not os.path.exists('./set_train') ):
    print('Descompactando dataset')
    with zipfile.ZipFile('./dataset/set_train.zip', 'r') as zip_ref:
        zip_ref.extractall('./')
    print('Dataset descompactado!')

caminho_resultado = Path('./resultado')
caminho_resultado.mkdir(exist_ok=True)

In [ ]:
caminho_imagens = "./set_train/"

from os import listdir
from os.path import isfile, join
pastas = [f for f in listdir(caminho_imagens) if not isfile(join(caminho_imagens, f))]
#pastas

#
# 0 neutra, 1 feliz, 2 triste, 3 surpreso, 4 bravo
# 
#
# 

In [ ]:
#Array que armazena o caminho completo para todos os arquivos
arquivos = []
# sentimentos = ['neutro','feliz','triste','surpreso','bravo']
sentimentos_lista = [0,1,2,3,4]
qnt_amostra = 0
sentimentos=[]

for pasta in pastas:
    caminho_completo = caminho_imagens+pasta+'/bmp/'
    pos_arq_atual = 0
    
    for f in listdir(caminho_completo):
        
        arquivos.append([caminho_completo+f])
        sentimentos.append(sentimentos_lista[pos_arq_atual])
        pos_arq_atual=pos_arq_atual+1
   
    
    pos_arq_atual = 0
    qnt_amostra = qnt_amostra+1
            
if len(arquivos) > 0:
    print('Encontrados: '+str(len(arquivos))+' arquivos')


In [ ]:
from imutils import face_utils
import dlib
import cv2
 
print('Detectando os pontos da face de todas as imagens')
# p = modelo pré-treinado para reconhecer 68 pontos da face
p = "./lib/shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)
imagens=[]

for x in range(0,len(arquivos)):
#     print (x)
    image = cv2.imread(arquivos[x][0])
    # image = cv2.imread('time.jpg')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    rects = detector(gray, 0)

    # Make the prediction and transfom it to numpy array
    shape = predictor(gray, rects[0])
    shape = face_utils.shape_to_np(shape)
    
    shape_transformado = shape.reshape(-1)
    imagens.append(shape_transformado)
    

print('Detecção concluida')    

In [ ]:
# Cria conjunto de treino e testes
from sklearn.model_selection import train_test_split


print('Separando amostra de treino e teste')

#Caso queira testar com outras amostras, basta alterar o random state
imagens_treino, imagens_teste, y_treino, y_teste = train_test_split(imagens, sentimentos, test_size=0.25, random_state=100)
print('Separação concluida com '+ str(len(y_treino))+' imagens de treino e '+str(len(y_teste))+' imagens de teste')

# Trecho usado para verificar o tamanho das amostras de treino e teste, com relação aos sentimentos
# Sim, tem formas mais eficientes de percorrer os arrays e selecionar, mas ainda estou me acostumando
# ao Pythom e tenho Prova de R esta semana =)

y_treino
neutro=0
feliz=0
triste=0
surpreso=0
bravo=0

for item in y_treino:
    if item==0 :
        neutro=neutro+1
    if item==1 :
        feliz=feliz+1
    if item==2 :
        triste=triste+1
    if item==3 :
        surpreso=surpreso+1
    if item==4 :
        bravo=bravo+1

print('Amostra treino:'+str(neutro))
print('Neutro:'+str(neutro))
print('Feliz:'+str(feliz))
print('Triste:'+str(triste))
print('Surpreso:'+str(surpreso))
print('Bravo:'+str(feliz))

y_treino
neutro=0
feliz=0
triste=0
surpreso=0
bravo=0

for item in y_teste:
    if item==0 :
        neutro=neutro+1
    if item==1 :
        feliz=feliz+1
    if item==2 :
        triste=triste+1
    if item==3 :
        surpreso=surpreso+1
    if item==4 :
        bravo=bravo+1
print()
print()
print('Amostra teste:'+str(neutro))
print('Neutro:'+str(neutro))
print('Feliz:'+str(feliz))
print('Triste:'+str(triste))
print('Surpreso:'+str(surpreso))
print('Bravo:'+str(feliz))
    

In [ ]:
# CNN Grid Search
# O Grid Search ajuda a encontrar hiperparametros de ajuste da CNN, com base
# em um conjunto pré-determinado de variações
# mostrando ao final quais ajustes produziram a melhor acuracidade do modelo
# 
# Obs: também funciona para outros tipos de modelos

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import numpy as np

sc = StandardScaler()
sc.fit(imagens_treino)

imagens_treino_pad = sc.transform(imagens_treino)
imagens_teste_pad = sc.transform(imagens_teste)


param_grid = [
        {
            'activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'solver' : ['lbfgs', 'sgd', 'adam'],
            'hidden_layer_sizes': [
             (1,),(2,),(3,),(4,),(5,),(6,),(7,),(8,),(9,),(10,),(11,), (12,),(13,),(14,),(15,),(16,),(17,),(18,),(19,),(20,),(21,)
             ]
        }
       ]

clf = GridSearchCV(MLPClassifier(), param_grid, cv=3,
                           scoring='accuracy')
clf.fit(imagens_treino_pad,y_treino)


In [ ]:
from joblib import dump, load

#Salva modelo e preparador de dados
dump(clf,'./resultado/MLPGridSearch.joblib')
dump(sc,'./resultado/sc_new.joblib')

y_saida = clf.predict(imagens_teste_pad)
print('Acuracia: %.2f' % accuracy_score(y_teste, y_saida))

In [ ]:
print("Melhores parametros:")
print(clf.best_params_)